In [7]:
import numpy as np
from skimage.io import imread
from scipy.spatial.distance import cdist
from PIL import Image
import time

In [100]:
IM_PATH = 'Photos//Full Size//Construction.png'
K = 20

In [101]:
im = imread(IM_PATH)

In [102]:
class K_means:
    def __init__(self, k, dist='euclidean', dim=3):
        self.K = k
        self.centers = None
        self.dist = dist
        self.dim = None
        
    def train(self, data, iterations):
        self.dim = data.shape[2]
        total_points = data.shape[0] * data.shape[1]
        data = np.reshape(data, (total_points, self.dim))
        self.centers = np.random.randint(0, 256, size=(self.K, self.dim))
        
        for _ in range(iterations):
            distances = cdist(data, self.centers, metric=self.dist)
            nearest_center = np.argmin(distances, axis=1) # -1 to get argmin instead of argmax
            
            for i in range(self.K):
                mask = nearest_center == i
                self.centers[i] = np.mean(data[mask], axis=0)
            self.centers = np.clip(self.centers, 1, 255)
                
    def transform(self, data):
        shape = data.shape
        total_points = shape[0] * shape[1]
        data = np.reshape(data, (total_points, self.dim))
        transform = {i: data[i] for i in range(self.centers.shape[0])}
        
        distances = cdist(data, self.centers, metric=self.dist)
        nearest_center = list(np.argmin(distances, axis=1))
        
        new_data = [transform[center] for center in nearest_center]
        return np.reshape(np.vstack(new_data), shape)

In [ ]:
model = K_means(K)
init = time.time()
model.train(im, 100)
final = time.time()

time_taken = final - init
print(time_taken)

In [ ]:
transform = model.transform(im)

In [ ]:
img = Image.fromarray(transform, 'RGB')

In [ ]:
img.save('test.png')